# A1111 Model Installer — SD 1.5 (UI)

Install **Stable Diffusion 1.5** models and common extension assets into the WebUI data directory.

**Folders (under `WEBUI_ROOT`, default `/workspace/a1111`):**
- **Checkpoints** → `models/Stable-diffusion/`
- **VAE** → `models/VAE/`
- **ControlNet (SD1.5)** → `models/ControlNet/`
- **ADetailer YOLO** → `models/adetailer/`
- **ESRGAN** → `models/ESRGAN/`
- **IP-Adapter** → `models/ControlNet/` (used via ControlNet)
- **AnimateDiff** → `models/AnimateDiff/`
- **ReActor (InsightFace)** → `models/insightface/` (create subfolders as needed)

Re-run any time — existing files are skipped.

> If a file is gated on Hugging Face, set `HF_TOKEN` in your RunPod template.

In [ ]:
# --- Paths / setup ---
from pathlib import Path
import os

ROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))
CKPT_DIR    = ROOT / "models/Stable-diffusion"
VAE_DIR     = ROOT / "models/VAE"
CN_DIR      = ROOT / "models/ControlNet"
AD_DIR      = ROOT / "models/adetailer"
ESRGAN_DIR  = ROOT / "models/ESRGAN"
IPAD_DIR    = CN_DIR                     # IP-Adapter weights live with ControlNet
    ANIM_DIR    = ROOT / "models/AnimateDiff"
REACT_DIR   = ROOT / "models/insightface"

for d in (CKPT_DIR, VAE_DIR, CN_DIR, AD_DIR, ESRGAN_DIR, IPAD_DIR, ANIM_DIR, REACT_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("Data root:", ROOT)
print("↳ Checkpoints:", CKPT_DIR)
print("↳ VAE:", VAE_DIR)
print("↳ ControlNet + IP-Adapter:", CN_DIR)
print("↳ ADetailer:", AD_DIR)
print("↳ ESRGAN:", ESRGAN_DIR)
print("↳ AnimateDiff:", ANIM_DIR)
print("↳ ReActor (InsightFace):", REACT_DIR)

## Helpers (progress + smart filenames)
Supports direct URLs (e.g. **Civitai**) with proper filename from headers, and **Hugging Face** repo/filename.

In [ ]:
# --- Helper functions ---
import os, re, shutil, requests
from tqdm import tqdm
from huggingface_hub import hf_hub_download
from pathlib import Path

def human(n: int) -> str:
    for u in ("B","KB","MB","GB"):
        if n < 1024: return f"{n:.1f} {u}"
        n /= 1024
    return f"{n:.1f} TB"

def _save_stream_to(dest: Path, r: requests.Response) -> Path:
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print("✔ Exists:", dest); return dest
    total = int(r.headers.get("Content-Length", 0))
    tmp = dest.with_suffix(dest.suffix + ".part")
    with open(tmp, "wb") as f, tqdm(total=total, unit="B", unit_scale=True, desc=dest.name, ncols=80) as bar:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk); bar.update(len(chunk))
    tmp.replace(dest)
    print(f"✔ Saved: {dest} ({human(dest.stat().st_size)})")
    return dest

def download_direct(url: str, dest_dir: Path) -> Path:
    r = requests.get(url, stream=True, allow_redirects=True)
    r.raise_for_status()
    cd = r.headers.get("Content-Disposition", "")
    m = re.search(r'filename="?([^";]+)"?', cd)
    if m:
        filename = m.group(1)
    else:
        filename = url.split("/")[-1].split("?", 1)[0]
        if not ("." in filename):
            filename += ".safetensors"
    return _save_stream_to(dest_dir/filename, r)

def download_to(url: str, dest: Path) -> Path:
    r = requests.get(url, stream=True, allow_redirects=True)
    r.raise_for_status()
    return _save_stream_to(dest, r)

def download_hf(repo_id: str, filename: str, dest_dir: Path, dest_name: str|None=None) -> Path:
    dest = dest_dir / (dest_name or filename)
    if dest.exists():
        print("✔ Exists:", dest); return dest
    print(f"→ HF: {repo_id}/{filename}")
    path = hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        token=os.environ.get("HF_TOKEN"),
        resume_download=True
    )
    shutil.copy2(path, dest)
    print(f"✔ HF saved: {dest} ({human(dest.stat().st_size)})")
    return dest

## Pick what to install
Each section has checkboxes **and** an **Extra URLs** box (one URL per line).

In [ ]:
import ipywidgets as W
from IPython.display import display, Markdown

# ---------- VAE ----------
vae_box = W.Checkbox(description="SD1.5 MSE VAE (stabilityai/sd-vae-ft-mse)", value=False)
vae_extra = W.Textarea(placeholder="Extra VAE URLs (.safetensors/.ckpt) — one per line", layout=W.Layout(width="100%", height="80px"))

# ---------- ControlNet (SD1.5) ----------
cn_items = [
    ("Canny",      "lllyasviel/control_v11p_sd15_canny"),
    ("SoftEdge",   "lllyasviel/control_v11p_sd15_softedge"),
    ("OpenPose",   "lllyasviel/control_v11p_sd15_openpose"),
    ("Depth",      "lllyasviel/control_v11p_sd15_depth"),
    ("LineArt",    "lllyasviel/control_v11p_sd15_lineart"),
    ("NormalBae",  "lllyasviel/control_v11p_sd15_normalbae")
]
cn_pre = {"Canny","SoftEdge","OpenPose"}
cn_checks = [W.Checkbox(description=label, value=(label in cn_pre)) for label,_ in cn_items]
cn_extra = W.Textarea(placeholder="Extra ControlNet URLs (.safetensors/.ckpt) — one per line", layout=W.Layout(width="100%", height="80px"))

# ---------- ADetailer YOLO ----------
ad_items = [
    ("face_yolov8n.pt",  "https://huggingface.co/Bing-su/adetailer/resolve/main/face_yolov8n.pt"),
    ("hand_yolov8n.pt",  "https://huggingface.co/Bing-su/adetailer/resolve/main/hand_yolov8n.pt"),
    ("face_yolov8s.pt",  "https://huggingface.co/Bing-su/adetailer/resolve/main/face_yolov8s.pt"),
    ("hand_yolov8s.pt",  "https://huggingface.co/Bing-su/adetailer/resolve/main/hand_yolov8s.pt")
]
ad_pre = {"face_yolov8n.pt","hand_yolov8n.pt"}
ad_checks = [W.Checkbox(description=name, value=(name in ad_pre)) for name,_ in ad_items]
ad_extra = W.Textarea(placeholder="Extra ADetailer model URLs (.pt) — one per line", layout=W.Layout(width="100%", height="80px"))

# ---------- ESRGAN ----------
esr_items = [
    ("4x-UltraSharp.pth", ("Coyote-A/4x-UltraSharp", "4x-UltraSharp.pth"))
]
esr_pre = {"4x-UltraSharp.pth"}
esr_checks = [W.Checkbox(description=name, value=(name in esr_pre)) for name,_ in esr_items]
esr_extra = W.Textarea(placeholder="Extra ESRGAN URLs (.pth) — one per line", layout=W.Layout(width="100%", height="80px"))

# ---------- IP-Adapter (SD1.5) ----------
# Lives in ControlNet folder; common weights from h94/IP-Adapter
ipad_items = [
    ("ip-adapter_sd15.safetensors",        ("h94/IP-Adapter", "models/ip-adapter_sd15.safetensors")),
    ("ip-adapter-plus_sd15.safetensors",   ("h94/IP-Adapter", "models/ip-adapter-plus_sd15.safetensors")),
    ("ip-adapter-faceid_sd15.safetensors", ("h94/IP-Adapter", "models/ip-adapter-faceid_sd15.safetensors"))
]
ipad_pre = {"ip-adapter_sd15.safetensors","ip-adapter-plus_sd15.safetensors"}
ipad_checks = [W.Checkbox(description=name, value=(name in ipad_pre)) for name,_ in ipad_items]
ipad_extra = W.Textarea(placeholder="Extra IP-Adapter URLs (.safetensors/.bin) — one per line", layout=W.Layout(width="100%", height="80px"))

# ---------- AnimateDiff ----------
anim_items = [
    ("mm_sd_v15_v2.ckpt", ("guoyww/animatediff", "mm_sd_v15_v2.ckpt"))
]
anim_pre = set()  # none pre-checked by default
    anim_checks = [W.Checkbox(description=name, value=(name in anim_pre)) for name,_ in anim_items]
anim_extra = W.Textarea(placeholder="Extra AnimateDiff URLs (.ckpt/.safetensors) — one per line", layout=W.Layout(width="100%", height="80px"))

# ---------- ReActor (InsightFace) ----------
# Paths expected by many guides:
#   models/insightface/models/ ...
#   models/insightface/antelopev2/ ...
REACT_MODELS_DIR   = REACT_DIR / "models"
REACT_ANTELOPE_DIR = REACT_DIR / "antelopev2"
REACT_MODELS_DIR.mkdir(parents=True, exist_ok=True)
REACT_ANTELOPE_DIR.mkdir(parents=True, exist_ok=True)

# (Leaving defaults unchecked; add your preferred links via Extra URLs.)
react_items = []
react_checks = []
react_extra = W.Textarea(placeholder="Extra InsightFace/antelopev2 URLs — one per line", layout=W.Layout(width="100%", height="80px"))

btn = W.Button(description="Download Selected", button_style="success", icon="download")
out = W.Output()

def _download_extras(text_widget, dest_dir):
    urls = [u.strip() for u in text_widget.value.splitlines() if u.strip()]
    for url in urls:
        try:
            download_direct(url, dest_dir)
        except Exception as e:
            print("✖", url, "→", e)

def on_click(_):
    out.clear_output()
    with out:
        errors = []
        # VAE
        if vae_box.value:
            try:
                download_hf("stabilityai/sd-vae-ft-mse", "vae-ft-mse-840000-ema-pruned.safetensors", VAE_DIR)
            except Exception as e:
                errors.append(("VAE", str(e)))
        _download_extras(vae_extra, VAE_DIR)

        # ControlNet
        for cb, (label, repo) in zip(cn_checks, cn_items):
            if not cb.value: continue
            try:
                download_hf(repo, "diffusion_pytorch_model.safetensors", CN_DIR, f"control_v11p_sd15_{label.lower()}.safetensors")
            except Exception as e:
                errors.append((f"ControlNet {label}", str(e)))
        _download_extras(cn_extra, CN_DIR)

        # ADetailer
        for cb, (name, url) in zip(ad_checks, ad_items):
            if not cb.value: continue
            try:
                download_to(url, AD_DIR / name)
            except Exception as e:
                errors.append((f"ADetailer {name}", str(e)))
        _download_extras(ad_extra, AD_DIR)

        # ESRGAN
        for cb, (name, hfref) in zip(esr_checks, esr_items):
            if not cb.value: continue
            try:
                repo, fname = hfref
                download_hf(repo, fname, ESRGAN_DIR)
            except Exception as e:
                errors.append((f"ESRGAN {name}", str(e)))
        _download_extras(esr_extra, ESRGAN_DIR)

        # IP-Adapter (into ControlNet dir)
        for cb, (name, (repo, fname)) in zip(ipad_checks, ipad_items):
            if not cb.value: continue
            try:
                download_hf(repo, fname, IPAD_DIR, name)
            except Exception as e:
                errors.append((f"IP-Adapter {name}", str(e)))
        _download_extras(ipad_extra, IPAD_DIR)

        # AnimateDiff
        for cb, (name, (repo, fname)) in zip(anim_checks, anim_items):
            if not cb.value: continue
            try:
                download_hf(repo, fname, ANIM_DIR)
            except Exception as e:
                errors.append((f"AnimateDiff {name}", str(e)))
        _download_extras(anim_extra, ANIM_DIR)

        # ReActor (InsightFace) — put extras under REACT_DIR; user can arrange subfolders
        for cb, _ in zip(react_checks, react_items):
            pass  # no defaults yet
        _download_extras(react_extra, REACT_DIR)

        print("\n=== Summary ===")
        if errors:
            for name, msg in errors:
                print("✖", name, "→", msg)
        else:
            print("✔ All downloads completed (check folders above).")

btn.on_click(on_click)

# --- Render UI ---
display(Markdown("### VAE"))
display(vae_box, vae_extra)

display(Markdown("### ControlNet (SD1.5)"))
for w in cn_checks: display(w)
display(Markdown("**Extra URLs**"))
display(cn_extra)

display(Markdown("### ADetailer models"))
for w in ad_checks: display(w)
display(Markdown("**Extra URLs**"))
display(ad_extra)

display(Markdown("### ESRGAN upscalers"))
for w in esr_checks: display(w)
display(Markdown("**Extra URLs**"))
display(esr_extra)

display(Markdown("### IP-Adapter (stored in ControlNet folder)"))
for w in ipad_checks: display(w)
display(Markdown("**Extra URLs**"))
display(ipad_extra)

display(Markdown("### AnimateDiff"))
for w in anim_checks: display(w)
display(Markdown("**Extra URLs**"))
display(anim_extra)

display(Markdown("### ReActor (InsightFace)"))
if not react_checks:
    display(Markdown("_No defaults preselected. Paste your InsightFace/antelopev2 links below._"))
display(Markdown("**Extra URLs**"))
display(react_extra)

display(btn, out)

## Restart WebUI (optional)
Use this after new files are added so A1111 refreshes lists.

In [ ]:
import subprocess, time, shlex
try:
    cmd = "supervisorctl -c /etc/supervisord.conf restart a1111"
    out = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
    if out.returncode == 0 and "a1111:" in (out.stdout or ""):
        print(out.stdout.strip())
    else:
        print("Supervisorctl unavailable, killing process…")
        subprocess.run(["pkill", "-f", "launch.py"], check=False)
        time.sleep(5)
        print("✔ WebUI terminated, Supervisor will auto-restart it")
    print("Wait 10–15 seconds, then refresh WebUI (7860).")
except Exception as e:
    print("⚠️ Restart failed:", e)
    print("Manual command:")
    print("supervisorctl -c /etc/supervisord.conf restart a1111")